# UK School Performance Data Mining

This is the first notebook in my *Analysis of UK School Performance* Exploratory Data Analysis (EDA) project. 

The focus of this notebook is to collect the data needed in order to analyse the effect of the proportion of students with English as an additional language (EAL) on the performance of schools in the UK.

I have obtained the *UK-Establishment-Data.csv* data set from the *gov.uk* website, which contains information for every educational establishment in the UK. However this data set does not contain any data on the proportion of pupils with EAL, nor the academic performance of the school. As a result, I will scrape the *gov.uk* website in order to obtain the relevant data.

## Obtaining the proportion of EAL students from one school

Before creating a script to collect data for each school in the UK, I will collect data about the proportion of EAL students for one primary school in the UK. For the sake of this notebook, I will be collect the data for Milton Road Primary School, Cambridge.

*NOTE: The gov.uk website blocks requests with a user agent of **python-requests**. As a result I needed to change the user agent to my user agent, however I have not added my user agent in this notebook. To run the cell below you must insert your user agent, which can be found with a Google search.*

In [1]:
# Below I import the necessary libraries.
import requests
from bs4 import BeautifulSoup

# Since the gov.uk website blocks requests with a user agent of python-requests, I have changed my user agent below (however I have not included
# it in this notebook). To make this code run, you must replace 'your user-agent' with your own user agent which can be found with a Google search.
headers = {'User-Agent': 'your user-agent'}

# I now request the page containing pupil population information for Milton Road Primary School
page = requests.get("https://www.compare-school-performance.service.gov.uk/school/133930/milton-road-primary-school/absence-and-pupil-population", headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')

# I now collect the data containing the percentage of students with English as a second language.
rows = soup.select("div #school-abspp-pupil-population-container table tr")
for iRowIndex in range(len(rows)):
    rowText = rows[iRowIndex].select(".text.label")[0]
    if rowText and rowText.contents[0].strip() == "Pupils whose first language is not English":
        row = rows[iRowIndex]

sPercentageEAL = row.select("td[headers='school']")[0].contents[0].strip()
fPercentageEAL = float(sPercentageEAL.strip("%"))

print(f"{fPercentageEAL}% of students at Milton Road Primary School have English as a second langauge.")

NameError: name 'row' is not defined

## Obtaining the proportion of EAL students for every school in the UK

I will now write an automated Python script to collect data about the proportion of EAL students for every school in the UK. In order to collect this data, I will iterate over every row of the *UK-Establishment-Data.csv* data set to obtain the school's name and URN. I will then scrape the relevant data for each school from the *gov.uk* website. 

*NOTE: The gov.uk website blocks requests with a user agent of **python-requests**. As a result I needed to change the user agent to my user agent, however I have not added my user agent in this notebook. To run the cell below you must insert your user agent, which can be found with a Google search.*

I will first import all of the necessary libraries and load the *UK-Establishment-Data.csv* data set into a pandas DataFrame object.

In [1]:
# Below I import the necessary libraries.
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

# I will first load the 'UK-Establishment-Data.csv* data set into a pd DataFrame object. 
dfEstabishmentData = pd.read_csv('UK-Establishment-Data.csv', encoding='latin-1')

# For the moment, we are only interested in each school's name and URN, so we will filter out all other columns. 
dfEstabishmentData = dfEstabishmentData[['EstablishmentName', 'URN']]


C:\Users\Adam\AppData\Local\Temp\ipykernel_23456\3277557559.py:9: DtypeWarning: Columns (31,38,46,48,50,51,55,72,73,81,86,87,88,89,90,91,92,93,94,95,96,117,120,121,125,126,129,139) have mixed types. Specify dtype option on import or set low_memory=False.
  dfEstabishmentData = pd.read_csv('UK-Establishment-Data.csv', encoding='latin-1')


I will now iterate through each row of my *dfEstablishmentData* DataFrame and scrape the necessary information for each school. For each school I will request the web page containing the percentage of EAL students enrolled in that school. In order to obtain the URL of this web page, I will use the general form of the *gov.uk* URL, which I found by exploring the web pages for various different schools. This general form can be seen below:

    https://www.compare-school-performance.service.gov.uk/school/school-URN/school-name/absence-and-pupil-population

Where *school-URN* and *school-name* should be replaced by the school's URN and name respectively. 

In order to increase the efficiency of our code I am going to implement paralelisation using the joblib library.

In [32]:
def getPercentageEALs(dfEstabishmentData):
    """
    Returns a pandas DataFrame containing the URN and percentage of EAL students for each school in the input DataFrame.

    This function is created to utilise the joblib library. 

    Parameters:
    ----------
    dfEstablishmentData: pandas DataFrame
                         The pandas DataFrame containing the name and URN for each school in the DataFrame.

    Returns:
    ----------
    dfMinedData: pandas DataFrame
                 A pandas DataFrame containing thu URN and percentage of students with EAL. For schools where this data is not recorded, the DataFrame
                 has the value NaN.


    """

    # Since the gov.uk website blocks requests with a user agent of python-requests, I will first change my user agent below (however I have not included
    # it in this notebook). To make this code run, you must replace 'your user-agent' with your own user agent which can be found with a Google search.
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}

    # I will now initialize a pd DataFrame which will store the data I am interested in.
    dfMinedData = pd.DataFrame()

    # I will now initialize two lists, which will store the data that I will add to the DataFrame 'dfMinedData'
    schoolURNs = []
    schoolPercentageEALs = []

    for iSchoolIndex in dfEstabishmentData.index:
        sName = "-".join(dfEstabishmentData.loc[iSchoolIndex, "EstablishmentName"].split()).lower()
        sURN = dfEstabishmentData.loc[iSchoolIndex, "URN"]

        schoolURNs.append(sURN)
        
        sURL = f"https://www.compare-school-performance.service.gov.uk/school/{sURN}/{sName}/absence-and-pupil-population"

        # I now request the page containing pupil population information for Milton Road Primary School
        page = requests.get(sURL, headers=headers)
        soup = BeautifulSoup(page.content, 'html.parser')

        # I now collect the data containing the percentage of students with English as a second language.
        rows = soup.select("div #school-abspp-pupil-population-container table tr")
        for iRowIndex in range(len(rows)):
            rowText = rows[iRowIndex].select(".text.label")[0]
            if rowText and rowText.contents[0].strip() == "Pupils whose first language is not English":
                row = rows[iRowIndex]
            
        if len(rows) == 0:
            schoolPercentageEALs.append('Incorrect URL')
            continue 
                
        sPercentageEAL = row.select("td[headers='school']")[0].contents[0].strip()

        if sPercentageEAL == "":
            schoolPercentageEALs.append(np.nan)
        else:
            schoolPercentageEALs.append(float(sPercentageEAL.strip("%")))

        if len(schoolURNs) != len(schoolPercentageEALs):
            print('not same length')

    dfMinedData['URN'] = schoolURNs
    dfMinedData['PercentageEAL'] = schoolPercentageEALs

    return dfMinedData


lSplitEstablishmentData = np.array_split(dfEstabishmentData, 8)

lMinedDataDFs = Parallel(n_jobs=8)(delayed(getPercentageEALs)(df) for df in lSplitEstablishmentData)

dfMinedData = pd.concat(lMinedDataDFs)

In [33]:
dfMinedData

,URN,PercentageEAL
0,100000,53.5
1,100001,NaN
2,100002,NaN
3,100003,NaN
4,100004,Incorrect URL
...,...,...
6308,402465,Incorrect URL
6309,402466,Incorrect URL
6310,402467,Incorrect URL
6311,402468,Incorrect URL


In [37]:
sum(dfMinedData['PercentageEAL'] == 'Incorrect URL')/len(dfMinedData)

0.46765922904828844

In [48]:
dfMinedData['PercentageEAL'].isna().sum()/len(dfMinedData)

0.11089112831376587


## Acknowledgements

I have scraped the *gov.uk* website in order to obtain data on the percentage of EAL students and performance of every school in the UK.

Contains public sector information licensed under the Open Government Licence v3.0.